In [1]:
import numpy as np
import matplotlib.pyplot as plt
plt.rcParams["font.family"] = "SimHei"
plt.rcParams["axes.unicode_minus"] = False
plt.rcParams["font.size"] = 12

from mpl_toolkits.mplot3d import Axes3D
%matplotlib qt

# 第一题

In [2]:
def fun(x1, x2):
    return 0.2 * (x1 + x2) ** 2 - 0.3 * x1 * x2 + 0.4


def grad_descent(f, x_init, eta=0.1, tol=1e-4, max_iter=1000, verbose=True):
    x_list = []
    y_list = []
    x1, x2 = x_init
    for i in range(max_iter):
        x_list.append((x1, x2))
        y = f(x1, x2)
        y_list.append(y)
        if len(y_list) >= 2 and y_list[-2] - y_list[-1] < tol:
            break
            
        delt = 1e-3
        g1 = (f(x1 + delt, x2) - f(x1, x2))/delt
        g2 = (f(x1, x2 + delt) - f(x1, x2))/delt
        x1 -= eta * g1
        x2 -= eta * g2
        
    if verbose:
        for index, (a, b) in enumerate(zip(x_list, y_list), start=1):
            print(f"第{index}次迭代，x={a}, y={b}")
    return x_list, y_list

x_list, y_list = grad_descent(fun, x_init=(4.8, 4.5), eta=0.01, verbose=False)


def plot_result(f, x, x_li, y_li):
    x1, x2 = x
    X1, X2 = np.meshgrid(x1, x2)
    X = np.array([X1.ravel(), X2.ravel()]).T
    y = f(X[:, 0], X[:, 1])
    Y = y.reshape(X1.shape)
    fig = plt.figure()
    ax = Axes3D(fig)
    surf = ax.plot_surface(X1, X2, Y, rstride=5, cstride=5, cmap="rainbow", alpha=0.8)
    # 将自变量的轨迹列表转换为ndarray数组，方便按列单独提取x1与x2的轨迹。
    array = np.asarray(x_li)
    x1_trace = array[:, 0]
    x2_trace = array[:, 1]
    ax.plot(x1_trace, x2_trace, y_list, c="b", ls="--", marker="o")
    ax.set_title("函数$y = 0.2(x1 + x2) ^ {2} - 0.3x1x2 + 0.4$")
    # 绘制三维图时，参数必须是数组类型，不支持标量，这点与绘制二维图不同。
    ax.plot(x1_trace[0:1], x2_trace[0:1], y_list[0:1], marker="*", ms=15, c="r", label="初始点")
    ax.plot(x1_trace[-1:], x2_trace[-1:], y_list[-1:], marker="*", ms=15, c="g", label="终止点")
    # 为曲面对象增加颜色条。
#     fig.colorbar(surf)
    ax.legend()
    # 创建新的画布，用来绘制等高线图。
    fig2 = plt.figure()
    ax2 = fig2.gca()
    m = ax2.contourf(X1, X2, Y, 10)
    ax2.scatter(x1_trace, x2_trace, c="r")
    # 为等高线图增加颜色条。
    fig2.colorbar(m)
    ax2.set_title("轨迹更新投影图")
    plt.show()

edge = 10
x1_domain = np.linspace(-edge, edge, 100)
x2_domain = np.linspace(-edge, edge, 100)
plot_result(fun, (x1_domain, x2_domain), x_list, y_list)

# 第二题&第三题

In [3]:
from sklearn.datasets import make_regression
from sklearn.model_selection import train_test_split
import numpy as np


X, y, coef = make_regression(n_samples=1000, n_features=5, bias=2.5, coef=True, noise=5, random_state=0)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=0)


class BatchGradDescent:
    """
    批量梯度下降类
    参数:
    max_iter:最大迭代次数
    eta: 学习率大小
    n: 连续n次没有提升就退出迭代
    """
    def __init__(self, max_iter=1000, eta=1e-3, n=5):
        self.max_iter = max_iter
        self.eta = eta
        self.n = n
    
    def fit(self, X, y):
        self.w = np.random.random((X.shape[1] + 1, 1))
        self.X = np.concatenate([np.ones((X.shape[0], 1)), X], axis=1)
        scores = []
        flag = False
        for itera in range(self.max_iter):
            for i in range(self.X.shape[1]):
                wi_gradient = 0
                # 这里使用的是批量梯度下降
                for j in range(0, len(self.X)):
                    x_single = self.X[j]
                    y_single = y[j]
                    wi_gradient += -(y_single - x_single.dot(self.w)[0]) * self.X[j][i]
                self.w[i][0] -= self.eta * wi_gradient
            scores.append(self.score(X, y))
            
            if len(scores) >= self.n + 1:
                for i in range(2, self.n + 2):
                    # 如果最近n次, 其中一次有提升就继续
                    if scores[-1] - scores[-i] > 0:
                        continue
                    # 如果都没有提升, 那就停止迭代
                    if i == self.n + 1:
                        flag = True
            if flag:
                break
        self.intercept = self.w.ravel()[0]
        self.coef = self.w[1:].ravel()
        
    
    def predict(self, Xtest):
        Xtest = np.concatenate([np.ones((Xtest.shape[0], 1)), Xtest], axis=1)
        return Xtest.dot(self.w).ravel()
    
    def score(self, X_test, Y_test):
        y_hat = self.predict(X_test)
        return self.r_square(y_hat, Y_test)
    
    def r_square(self, y_hat, Y_test):
        rss = np.sum(np.square(Y_test - y_hat))
        tss = np.sum(np.square(Y_test - y_hat.mean()))
        return 1 - rss/ tss

bgd = BatchGradDescent(max_iter=1000, eta=1e-4, n=5)
bgd.fit(X_train, y_train)
print(bgd.intercept, bgd.coef)

# 真实的系数
print("真实的系数:", coef)

print(bgd.score(X_train, y_train))
print(bgd.score(X_test, y_test))

2.512066578851029 [41.06483863 66.50721497 10.9424458  60.43148722 25.69384343]
真实的系数: [41.20593377 66.49948238 10.71453179 60.19514224 25.96147771]
0.9972551641628642
0.9978527244815283
